# Word2Vec 모델을 통해 증상별 질병 찾기

# 관련 모듈 설치

In [2]:
!pip install pandas

  Using cached tzdata-2024.1-py2.py3-none-any.whl.metadata (1.4 kB)
   ---------------------------------------- 0.0/10.8 MB ? eta -:--:--
   ---------------------------------------- 0.1/10.8 MB 871.5 kB/s eta 0:00:13
   -- ------------------------------------- 0.6/10.8 MB 7.2 MB/s eta 0:00:02
   ---- ----------------------------------- 1.2/10.8 MB 8.8 MB/s eta 0:00:02
   ------ --------------------------------- 1.8/10.8 MB 9.4 MB/s eta 0:00:01
   -------- ------------------------------- 2.3/10.8 MB 9.7 MB/s eta 0:00:01
   ---------- ----------------------------- 2.7/10.8 MB 9.7 MB/s eta 0:00:01
   ------------ --------------------------- 3.3/10.8 MB 10.0 MB/s eta 0:00:01
   -------------- ------------------------- 3.8/10.8 MB 10.2 MB/s eta 0:00:01
   ---------------- ----------------------- 4.4/10.8 MB 10.3 MB/s eta 0:00:01
   ------------------ --------------------- 4.9/10.8 MB 10.4 MB/s eta 0:00:01
   -------------------- ------------------- 5.6/10.8 MB 10.9 MB/s eta 0:00:01
   -----

In [3]:
!pip install numpy

In [4]:
pip install konlpy

  Using cached konlpy-0.6.0-py2.py3-none-any.whl.metadata (1.9 kB)
Using cached konlpy-0.6.0-py2.py3-none-any.whl (19.4 MB)
   ---------------------------------------- 0.0/351.7 kB ? eta -:--:--
   - -------------------------------------- 10.2/351.7 kB ? eta -:--:--
   ------ --------------------------------- 61.4/351.7 kB 1.1 MB/s eta 0:00:01
   ---------------------------------------- 351.7/351.7 kB 3.6 MB/s eta 0:00:00
   ---------------------------------------- 0.0/3.8 MB ? eta -:--:--
   ------ --------------------------------- 0.6/3.8 MB 17.6 MB/s eta 0:00:01
   ----------- ---------------------------- 1.1/3.8 MB 12.0 MB/s eta 0:00:01
   ------------------- -------------------- 1.9/3.8 MB 11.8 MB/s eta 0:00:01
   ------------------------- -------------- 2.4/3.8 MB 11.7 MB/s eta 0:00:01
   ------------------------------ --------- 2.9/3.8 MB 11.8 MB/s eta 0:00:01
   ------------------------------------ --- 3.5/3.8 MB 11.7 MB/s eta 0:00:01
   --------------------------------------- 

In [5]:
pip install gensim

     ---------------------------------------- 0.0/59.0 kB ? eta -:--:--
     -------------------------- ----------- 41.0/59.0 kB 991.0 kB/s eta 0:00:01
     ---------------------------------------- 59.0/59.0 kB 1.0 MB/s eta 0:00:00
  Using cached smart_open-7.0.4-py3-none-any.whl.metadata (23 kB)
   ---------------------------------------- 0.0/24.0 MB ? eta -:--:--
    --------------------------------------- 0.5/24.0 MB 10.9 MB/s eta 0:00:03
   -- ------------------------------------- 1.2/24.0 MB 13.1 MB/s eta 0:00:02
   --- ------------------------------------ 1.9/24.0 MB 12.4 MB/s eta 0:00:02
   ---- ----------------------------------- 2.4/24.0 MB 12.0 MB/s eta 0:00:02
   ---- ----------------------------------- 3.0/24.0 MB 11.8 MB/s eta 0:00:02
   ----- ---------------------------------- 3.5/24.0 MB 11.7 MB/s eta 0:00:02
   ------ --------------------------------- 4.0/24.0 MB 11.7 MB/s eta 0:00:02
   ------- -------------------------------- 4.6/24.0 MB 11.7 MB/s eta 0:00:02
   -----

- 오류
- 파이썬 3.12버전에서는 gensim 설치시 오류 발생
- 3.8버전으로 환경 설정

# 관련 모듈 임포트

In [6]:
import pandas as pd
from gensim.models import Word2Vec
from konlpy.tag import Okt  # 한국어 처리를 위한 konlpy 라이브러리
import numpy as np

# 데이터 준비

In [13]:
# CSV 파일 경로
file_path = './../Data/diseases_240510.csv'

# CSV 파일 읽기
df = pd.read_csv(file_path)
df = df.dropna(subset=['symptoms'])

# 결과를 담을 리스트 초기화
result_lists = []

# 각 행(질병)에 대해 반복
for index, row in df.iterrows():
    disease_name = row['disease_name']
    symptoms = row['symptoms'].split(', ')
    
    # 첫 번째 형식의 리스트 생성  
    list1 = []
    for symptom in symptoms:
        list1.extend([disease_name, symptom])
    
    # 두 번째 형식의 리스트 생성
    list2 = [disease_name] + symptoms
    
    # 결과 리스트에 추가
    result_lists.append(list1)
    result_lists.append(list2)

In [14]:
result_lists

[['RS바이러스 감염증(RS virus infection)',
  '기침',
  'RS바이러스 감염증(RS virus infection)',
  '가래',
  'RS바이러스 감염증(RS virus infection)',
  '열',
  'RS바이러스 감염증(RS virus infection)',
  '빈호흡',
  'RS바이러스 감염증(RS virus infection)',
  '콧물',
  'RS바이러스 감염증(RS virus infection)',
  '코막힘',
  'RS바이러스 감염증(RS virus infection)',
  '청색증'],
 ['RS바이러스 감염증(RS virus infection)',
  '기침',
  '가래',
  '열',
  '빈호흡',
  '콧물',
  '코막힘',
  '청색증'],
 ['각막염(Keratitis)',
  '안구혼탁',
  '각막염(Keratitis)',
  '눈의 충혈',
  '각막염(Keratitis)',
  '눈물흘림',
  '각막염(Keratitis)',
  '눈의 통증',
  '각막염(Keratitis)',
  '시력 감소',
  '각막염(Keratitis)',
  '눈부심'],
 ['각막염(Keratitis)', '안구혼탁', '눈의 충혈', '눈물흘림', '눈의 통증', '시력 감소', '눈부심'],
 ['간농양(Liver abscess)',
  '황달',
  '간농양(Liver abscess)',
  '기운없음',
  '간농양(Liver abscess)',
  '오한',
  '간농양(Liver abscess)',
  '열',
  '간농양(Liver abscess)',
  '복부 통증',
  '간농양(Liver abscess)',
  '오심',
  '간농양(Liver abscess)',
  '설사',
  '간농양(Liver abscess)',
  '압통',
  '간농양(Liver abscess)',
  '구토',
  '간농양(Liver abscess)',
  '간비대'],
 ['간농양(Liver a

In [15]:
# 질병 리스트와 증상 리스트 초기화
disease_list = []
symptom_list = []

# 각 행(질병)에 대해 반복
for index, row in df.iterrows():
    # 질병명 추가
    disease_name = row['disease_name']
    if disease_name not in disease_list:
        disease_list.append(disease_name)
    
    # 증상 추가
    symptoms = row['symptoms'].split(', ')
    for symptom in symptoms:
        if symptom not in symptom_list:
            symptom_list.append(symptom)

In [17]:
len(disease_list), len(symptom_list)

(1199, 844)

In [21]:
symptom_list

['기침',
 '가래',
 '열',
 '빈호흡',
 '콧물',
 '코막힘',
 '청색증',
 '안구혼탁',
 '눈의 충혈',
 '눈물흘림',
 '눈의 통증',
 '시력 감소',
 '눈부심',
 '황달',
 '기운없음',
 '오한',
 '복부 통증',
 '오심',
 '설사',
 '압통',
 '구토',
 '간비대',
 '피부염증',
 '발진',
 '열감',
 '환부 부종',
 '목의 통증',
 '삼키기 곤란',
 '두통',
 '안면홍조',
 '권태감',
 '식욕부진',
 '정신 지체',
 '태아 저체중',
 '무증상',
 '청력장애',
 '난청',
 '궤양',
 '피로감',
 '눈꼽',
 '눈의 자극감',
 '눈의 피로',
 '객혈',
 '체중감소',
 '근육경련',
 '뼈의 변형',
 '관절통',
 '복부팽만감',
 '빈맥',
 '발한',
 '복수',
 '시야장애',
 '눈운동신경마비',
 '혼수',
 '졸림',
 '목 주변 부종',
 '고름',
 '피부궤양',
 '통증 없는 부종',
 '발가락 사이 갈라짐',
 '손,발톱이 갈라짐',
 '피부 각질',
 '피부소양감',
 '뇌염증상',
 '뇌전증 발작',
 '호흡곤란',
 '근육강직',
 '치매',
 '이상행동',
 '기억장애',
 '입 주변 부종',
 '입술 갈라짐',
 '소양감',
 '작열감',
 '물집',
 '피부색 변화',
 '근육통',
 '감기 증상',
 '피부 긴장도 저하',
 '탈수',
 '악취',
 '배뇨곤란',
 '비정상적 질 분비물',
 '배뇨 시 통증',
 '경부 림프절병증',
 '비정상적 질출혈',
 '복부 불편감',
 '경련',
 '성격의변화',
 '경부 강직',
 '정신 혼미',
 '의식 저하',
 '방향감각 상실',
 '림프 부종',
 '환부 통증',
 '의식 변화',
 '언어장애',
 '딱지',
 '반점',
 '보챔',
 '체모감소',
 '눈이 무거운 느낌',
 '박동성 통증',
 '뇌압상승증상',
 '가슴 통증',
 '결막염',
 '얼굴부종',
 '단백뇨',
 '수면장애',
 '신

# 모델 학습 및 결과

In [18]:
# 입력 증상 - 단어로 입력해야함
input_text =  "기침"

# 예시 데이터
sentences = result_lists

# 모델 학습
model = Word2Vec(sentences, vector_size=150, window=5, min_count=5, workers=4, sg=1)

# '열'에 대한 상위 5개 유사 단어 찾기
similar_words = model.wv.most_similar(input_text, topn=300)

# 질병 데이터만 추출하기 위한 질병 목록 (예시)
diseases = disease_list

# 질병 목록에 있는 단어들만 필터링하여 저장
disease_similar_words = [(word, similarity) for word, similarity in similar_words if word in disease_list][:5]

print(disease_similar_words)

[('후두암(Larynx  cancer)', 0.9462725520133972), ('병원 감염성 폐렴(Hospital acquired pneumonia (HAP))', 0.9445509910583496), ('속립성 결핵(Miliary tuberculosis)', 0.9410330653190613), ('코로나-19(COVID-19)', 0.9405409693717957), ('결핵(Tuberculosis)', 0.9359949827194214)]


# 문장형 도전
더 정확한 결과를 위해 수정 중

In [44]:
# Okt 형태소 분석기 초기화
okt = Okt()

# 분석할 문장
sentence_input = "두통이 너무 심해서 몸살이 난 것 같아 기침이 난다"

# 문장을 형태소 분석하여 토큰화
tokens = okt.morphs(sentence_input)

# 모델과 질병 데이터를 이용해 문장의 토큰 중에 질병 목록에 있는 단어들과 가장 유사한 단어 찾기
similar_diseases = []

for token in tokens:
    if token in model.wv.key_to_index:  # 모델이 학습한 단어인지 확인
        # 해당 토큰과 유사한 단어들 중에서 질병 목록에 있는 단어들만 추출
        similar_words = model.wv.most_similar(token, topn=300)
        for word, similarity in similar_words:
            if word in diseases:  # 질병 목록에 있는 단어인지 확인
                similar_diseases.append((word, similarity))
                break  # 가장 유사한 질병 하나만 추가하고 다음 토큰으로 넘어감
        print(token)

# 유사한 질병들 중 유사도가 높은 순으로 정렬
similar_diseases = sorted(similar_diseases, key=lambda x: x[1], reverse=True)

# 가장 유사한 질병 5개 출력
print(similar_diseases[:5])


두통
몸살
기침
[('말기 신질환(End stage renal disease)', 0.9946784973144531), ('바이러스성 뇌수막염(Viral meningitis)', 0.9778353571891785), ('후두암(Larynx  cancer)', 0.9454103708267212)]


In [32]:
from gensim.models import Word2Vec
from collections import defaultdict

# 입력 증상 - 여러 개의 증상을 리스트로 입력
input_symptoms = ["두통", "기침"]

# 예시 데이터
sentences = result_lists

# 모델 학습
model = Word2Vec(sentences, vector_size=150, window=5, min_count=5, workers=4, sg=1)

# 질병 데이터만 추출하기 위한 질병 목록 (예시)
diseases = disease_list

# 각 증상에 대한 유사 질병을 저장하는 사전 초기화
symptom_disease_similarity = defaultdict(list)

# 각 증상에 대해 유사한 질병 찾기
for symptom in input_symptoms:
    similar_words = model.wv.most_similar(symptom, topn=300)
    for word, similarity in similar_words:
        if word in diseases:
            symptom_disease_similarity[symptom].append((word, similarity))

# 모든 증상과 연관된 질병들의 교집합 찾기
common_diseases = set(symptom_disease_similarity[input_symptoms[0]]).intersection(
    *[set(symptom_disease_similarity[symptom]) for symptom in input_symptoms[1:]]
)

# 교집합에서 유사도를 기준으로 상위 5개 질병 추출
sorted_common_diseases = sorted(common_diseases, key=lambda x: x[1], reverse=True)[:5]

print(sorted_common_diseases)

[]


In [36]:
input_symptoms = ["두통", "기침"]

# 각 증상에 대해 가장 유사한 질병 찾기
symptom_to_diseases = defaultdict(list)
# 각 증상에 대해 유사한 질병 찾기
for symptom in input_symptoms:
    similar_words = model.wv.most_similar(symptom, topn=300)
    for word, similarity in similar_words:
        if word in diseases:
            symptom_to_diseases[symptom].append((word, similarity))

# 모든 증상과 관련된 질병들의 교집합 찾기
common_diseases = set(symptom_to_diseases[input_symptoms[0]])
for symptom in input_symptoms[1:]:
    common_diseases.intersection_update(symptom_to_diseases[symptom])

# 교집합에서 유사도를 기준으로 상위 5개 질병 추출 (단순화를 위해 이 부분은 구현하지 않음)
# 실제 구현 시에는 각 질병에 대한 유사도 점수를 계산하고 상위 5개를 선택해야 함

# 결과 출력
print(f"입력된 증상 {input_symptoms}에 대한 공통 질병:", list(common_diseases)[:5])

입력된 증상 ['두통', '기침']에 대한 공통 질병: []


In [48]:
symptom_to_diseases[input_symptoms[0]][1:2]

[('박테리아성 뇌수막염(Bacterial meningitis)', 0.9697558879852295)]

In [55]:
input_symptoms = ["가려움", "기침"]

# 각 증상에 대해 가장 유사한 질병과 그 유사도 찾기
symptom_to_diseases_similarity = {}
for symptom in input_symptoms:
    similar_words = model.wv.most_similar(symptom, topn=300)
    symptom_to_diseases_similarity[symptom] = [(word, similarity) for word, similarity in similar_words if word in diseases]

# 모든 증상과 관련된 질병들의 교집합 찾기
common_diseases = set([disease for disease, _ in symptom_to_diseases_similarity[input_symptoms[0]]])
for symptom in input_symptoms[1:]:
    common_diseases.intersection_update([disease for disease, _ in symptom_to_diseases_similarity[symptom]])

# 교집합에 있는 질병들의 증상별 유사도 정보와 평균 유사도 계산
disease_similarity_info = []
for disease in common_diseases:
    similarities = []
    for symptom in input_symptoms:
        # 해당 증상과 질병의 유사도 찾기
        similarity = [sim for dis, sim in symptom_to_diseases_similarity[symptom] if dis == disease]
        if similarity:  # 유사도 정보가 있으면 추가
            similarities.append(similarity[0])
    
    # 평균 유사도 계산
    if similarities:  # 유사도 정보가 하나라도 있으면
        avg_similarity = np.mean(similarities)
        disease_similarity_info.append((disease, avg_similarity, similarities))

# 유사도 평균이 높은 순으로 정렬 후 상위 10개 출력
disease_similarity_info.sort(key=lambda x: x[1], reverse=True)
top_diseases = disease_similarity_info[:10]

# 결과 출력
for disease, avg_similarity, similarities in top_diseases:
    print(f"질병: {disease}, 평균 유사도: {avg_similarity:.2f}, 증상별 유사도: {similarities}")


KeyError: "Key '가려움' not present in vocabulary"

In [50]:
intersection_diseases

[]